#Jointly Optimized Regressors for Image Super-resolution
论文地址：http://people.ee.ethz.ch/~daid/JOR/

Abstract：
1. 训练过程中产生联合分辨率的最小误差
2. 测试过程中，为每一个patch选择最合适的回归器，相同的patch由低分辨率到高分辨率的过程一样，可以是用kNN聚类

Introduce：
1. 图像尺寸的改变是最常见的操作，缩小（downsampling）图像尺寸是很容易的，然而图像的放大（upsampling）却存在很大问题：图像相应的二义性（the notorious ambiguity of patch
Correspondence）-- a low-resolution (LR) image patch canbe the down-sampled version of enormous high-resolution(HR) patches.

2. 由于需要降低歧义性，所以需要各种形式的先验知识。（In order to reduce the ambiguity, different forms of prior knowledge have been explored）先验举例EG：图像足够的平滑。

3. 基于范本的超分辨率算法，我们把每一个图像分别的提升其分辨率，应用已知/的LR和HR对应模式。Patch重叠的部分求均值。

4. 回归函数是高度非线性的，产生一个可以用在每一个patch上合适的非线性函数是很困难的。[TDV13,YY13] 文章中，为了解决复杂的非线性问题，通过应用一系列局部函数的集合。本文扩展这个方法，通过提出一个新颖的方法：一系列局部回归函数联合优化。

5. 在实际应用中，我们联合的优化一系列局部回归函数。对于训练数据，每一个LR都和是找到可以产生和HR误差最小的回归函数，也就是可以找到最合适的回归函数。然而对于测试数据，LR已知，HR却未知。为了解决这个问题，我们回到训练数据集，外推出局部领域的信息。

Previos Work
1.	最早使用的方法：The oldest direction and very popular in commercial software is represented by the data invariant linearfilters. Nearest-Neighbor, Bilinear, Bicubic, Hamming,NEDI, or Lanczos interpolation kernels are among the bestknown [Duc79,TBU00,LO01].

2.	产生的问题：exhibit visual artifacts such as blurring, ringing, blocking, aliasing.为了解决这些方法带来的问题，就需要足够的先验知识。（stronger prior information.）

3.	另一个方向的方法:使用不同类型图像的能量函数分布的具体形式 的先验知识。
EG：
[TRF03] the sparse derivative priors are exploited
[TD05] the regularization PDE’s
[DHX_07] the edge smoothness prior
[Fat07] the edge statistics are enforced to obtain the HR solution
[EGA_13, MI13] try to estimate appropriate blur kernels instead finding good image priors
大多数的方法工作在小的patch上然后再合成大的高清patch

4.	第三类方法:基于实例大多数方法分别处理高清图像高低频率的部分。
对于低频部分：可以通过快速的插值核（双线性插值）进行合理的计算。
对于高频部分：图像就化简为了细节估计问题。
使用这个先验的方法:建立HR-LR采样对，寻找输入patch最接近的trainingLR。

5.	局部自相似、局部图象冗余在[FF11] and [GBI09].中用到。这些方法不许要使用额外的信息

6.	稀疏编码 的系数假设

7. 新提出的方法和xneighbor embedding methods相似，新颖的地方在于使用了联合最优化的回归器。

#Approach
1. 联合学习一组固定的回归器。共同提供最小的回归误差；选择最合适的回归器进行线性回归。

2. 由于现实世界的patch多种多样所以很难找到最合适的全局patch，所以我们训练一组典型的回归器。联合起来能够给训练数据提供最小的超分辨率误差。

3. DataCollection 给出一系列的训练图像作为高清的图像，他们的下采样版本作为低分辨率的图像。一般下采样的三到四倍（固定的缩放比例）。随后将所有的LR图像分解成small patch x，然后寻找他们对应的HRpatch y来构成训练数据集合。

4. Training ：获得训练数据集之后，我们的目标就变成了寻找一组固定的O个回归器$F={f_0,f_1...,f_O}$，但是需要回归的空间很大，我们需要筛选相互独立、互相补充的数据。本文通过聚类的方法解决这个问题：同一个聚类的数据可以使用相同的回归器（O个回归器当中的一个）进行计算。我们并不约束在相同聚类中的patch使用相同的样式。patch能够共享回归器函数，即使在特征空间有很大差距。

5. 问题的数学定义：将数据D聚类成O个类，然后通过O个类训练O个回归器。类别o和回归函数$f_o$是相关联的。并且拥有一个指示器向量$c\in \{0,1\}^L$,实例的个数是L，主要用于分辨实例l是否属于类别o，每一个类都有一个指示器。Hard cluster 是每一个patch只属于一个类别。我们的目标就是优化如下的方程
$$min_{C,F}\sum_{l=1}^L\sum_{o=1}^Oc_{o,l}\Vert f_o(x_l)-y_l\Vert^2$$

6. 以上公式C、F都是未知的不能同时进行优化。所以我们使用EM算法，来迭代的求解这个问题。 E-step：先给出$c_o$。计算回归器$f_o$；M-step：拥有回归器$f_o$之后我们在优化$c_o$，我们交替的使用E-MSteps知道算法收敛（convergence）。这里使用的E-step是不准确的，主要由于E-step并没有使用概率期望。而初始化的$c_o$主要是通过在L个LRpatch上运行K-means聚类方法得到的。
![Git Commands](1.png)

对于回归函数的解释：
使用Ridge Regression。使用岭回归：
得到的目标$f_o$为投影函数：
$$P_o = Y_o(X_o^tX_o+\lambda I)^{-1}X_o^T$$
Y表示HRpatch向量化堆叠结果，X表示LRpatch向量化堆叠结果，$\lambda$表示调整参数固定为0.1

岭回归目的：The identity matrix is added mainly to regularize the solution, i.e. to avoid overﬁtting, which is derived from the regularization term of Ridge Regression

通过以上结果，LR的超分辨率问题变转换为投影问题：
$$y_{o,l}=f_o(x_l)=P_ox_l$$
超分辨率误差为；$$z_{x,l}=\Vert y_{o,l}'-y_{o,l}\Vert$$

Z就记录了patch x通过回归器o计算后的误差，大小问（l,o）

###进行上采样，With JOR
训练结束之后HRpatch会被丢弃。每一个低分辨率的x都会和一个$z_l$向量相联系，包好 $z_l$当中包含了O个值，表示了这个LRpatch $x_l$使用每一个回归器o计算所产生的误差。

在测试的过程中，我们将LR image分解成small patches（大小和训练数据集的尺寸一样），然后给每一个LRpatch赋予一个最合适的回归器，然后通过投影得到LR对应的HR。最后通过HRpatches重构出想要的HRimage。

如何选择合适的回归器：

最有回归器的选择过程有两步：
1. 所有的超分辨率回归误差已经完成计算
2. 将拥有最小回归误差的回归器用作超分辨率计算。

kmeans聚类中的邻居反应他的

###岭回归
数值计算方法的“稳定性”是指在计算过程中舍入误差是可以控制的。
回归分析中常用的最小二乘法是一种无偏估计：
$$XB=Y$$
当X列满秩时，有
$$B=X^-Y=(X^TX)^{-1}X^TY$$
当X不是列满秩，或者某些列之间的线性相关性比较大时，$X^TX$的行列式接近于0，即$X^TX$接近于奇异，计算$(X^TX)^{-1}$时误差会很大。此时传统的最小二乘法缺乏稳定性与可靠性。

岭回归是对最小二乘回归的一种补充，它损失了无偏性，来换取高的数值稳定性，从而得到较高的计算精度。
$$B(k) =(X^TX+kl)^{-1}X^TY$$

随着k的增大，B(k)中各元素bi(k)的绝对值均趋于不断变小，它们相对于正确值bi的偏差也越来越大。k趋于无穷大时，B(k)趋于0。b(k)随k的改变而变化的轨迹，就称为岭迹。实际计算中可选非常多的k值，做出一个岭迹图，看看这个图在取哪个值的时候变稳定了，那就确定k值了。

X不满足列满秩，换句话就是说样本向量之间具有高度的相关性（如果每一列是一个向量的话）。遇到列向量相关的情形，岭回归是一种处理方法，也可以用主成分分析PCA来进行降维。

##这篇论文应用了重要的方法：
Anchored neighborhood regression for fast example-based super resolution.